In [139]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from keras.layers import Activation, Dense
import pandas as pd 
from tensorflow import keras

df = pd.read_csv("heart_disease_health_indicators_BRFSS2015.csv")
df.head()

,HeartDiseaseorAttack,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,Diabetes,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0


In [140]:
df = df.drop(columns = ['GenHlth', 'MentHlth','PhysHlth','Income','Education','Age'])
df.columns

Index(['HeartDiseaseorAttack', 'HighBP', 'HighChol', 'CholCheck', 'BMI',
       'Smoker', 'Stroke', 'Diabetes', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk', 'Sex'],
      dtype='object')

In [141]:
dummies = pd.get_dummies(df)

In [142]:
y = dummies['HeartDiseaseorAttack'].values
X = dummies.drop(columns='HeartDiseaseorAttack').values

In [143]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [164]:
scaler = StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

array([ 1.15497318,  1.16628632,  0.19682794,  0.54662825, -0.89288603,
        4.85803103,  2.44375473,  0.56682158,  0.759569  , -2.07939751,
       -0.24408668,  0.22726258, -0.30248534,  2.2256741 ,  1.12820509])

In [145]:
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(Dense(units=64, activation = "tanh", input_dim= len(X_train[0])))

# Second hidden layer
nn.add(Dense(units=32, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 64)                1024      
                                                                 
 dense_10 (Dense)            (None, 32)                2080      
                                                                 
 dense_11 (Dense)            (None, 1)                 33        
                                                                 
Total params: 3137 (12.25 KB)
Trainable params: 3137 (12.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [146]:
nn.compile(loss="binary_crossentropy", optimizer= "adam", metrics= ["accuracy"])

In [147]:
nn_fitted = nn.fit(X_train_scaled,y_train, epochs=100)

Epoch 1/100
5946/5946 [==============================] - 5s 782us/step - loss: 0.2618 - accuracy: 0.9053
Epoch 2/100
5946/5946 [==============================] - 5s 784us/step - loss: 0.2571 - accuracy: 0.9067
Epoch 3/100
5946/5946 [==============================] - 5s 788us/step - loss: 0.2567 - accuracy: 0.9067
Epoch 4/100
5946/5946 [==============================] - 5s 796us/step - loss: 0.2563 - accuracy: 0.9066
Epoch 5/100
5946/5946 [==============================] - 5s 791us/step - loss: 0.2561 - accuracy: 0.9065
Epoch 6/100
5946/5946 [==============================] - 5s 794us/step - loss: 0.2560 - accuracy: 0.9068
Epoch 7/100
5946/5946 [==============================] - 5s 807us/step - loss: 0.2558 - accuracy: 0.9069
Epoch 8/100
5946/5946 [==============================] - 5s 821us/step - loss: 0.2556 - accuracy: 0.9068
Epoch 9/100
5946/5946 [==============================] - 5s 798us/step - loss: 0.2555 - accuracy: 0.9068
Epoch 10/100
5946/5946 [==============================]

In [148]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1982/1982 - 1s - loss: 0.2675 - accuracy: 0.9061 - 1s/epoch - 638us/step
Loss: 0.2674786150455475, Accuracy: 0.9061495065689087


In [149]:
nn.save("Heart_Disease_Pred.h5")

C:\Users\nitsu\anaconda3\envs\ml\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [150]:
reloded_Heart_Disease_Pred = keras.models.load_model("Heart_Disease_Pred.h5")

In [218]:
new_data = [[1, 0, 1, 30,
       0, 0, 0, 1, 1, 1,
       0, 1, 0, 0, 0]]

In [219]:
new_data_df = pd.DataFrame(new_data, columns=['HighBP', 'HighChol', 'CholCheck', 'BMI',
       'Smoker', 'Stroke', 'Diabetes', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk', 'Sex'])
new_data_scaled = scaler.fit(X_train).transform(new_data_df)

C:\Users\nitsu\anaconda3\envs\ml\Lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [220]:
prediction = reloded_Heart_Disease_Pred.predict(new_data_scaled)

1/1 [==============================] - 0s 12ms/step


In [221]:
print(f"prediction: {prediction}")

prediction: [[0.03309385]]
